In [7]:
# Carica il file csv e coonverto colonna dei prezzi in int
import pandas as pd
import re
import itertools

df=pd.read_csv("milano_dati_booking.csv")
df['prezzo'] = df['prezzo'].apply(lambda x: int(re.search(r'\d+', x).group()))
df.columns

Index(['nome_hotel', 'prezzo', 'stanza', 'città', 'data', 'punteggio',
       'numero_recensioni', 'distanza_centro', 'genius', 'offerte',
       'colazione_inclusa', 'senza_pagamento_anticipato',
       'cancellazione_gratuita', 'os', 'username', 'user_agent_type',
       'TIMESTAMP'],
      dtype='object')

In [8]:
#Prendo tutte le combinazioni di coppie di ricerche sulla stessa città
df_list = df['TIMESTAMP'].unique()
coppie_df = list(itertools.combinations(df_list, 2))
len(coppie_df)


45

In [9]:
# Creo un Dataframe per vedere differenze di prezzo tra diverse ricerche
list_dict_df = []
for i in coppie_df:
    orario1 = i[0]
    orario2 = i[1]
    df1 = df[df["TIMESTAMP"]==orario1]
    df2 = df[df["TIMESTAMP"]==orario2]
    df1 = df1.drop_duplicates()
    df2 = df2.drop_duplicates()
    if df1['user_agent_type'].unique()[0]==df2['user_agent_type'].unique()[0]:
        merged_df = pd.merge(df1, df2, on=['nome_hotel','stanza','data'], how='inner')
    else:
        merged_df = pd.merge(df1, df2, on=['nome_hotel','data'], how='inner')
    initial_len = len(merged_df)
    merged_df = merged_df[merged_df["prezzo_x"]!=merged_df["prezzo_y"]]
    # Elimino hotel con stesso nome 
    righe_droppate = len(merged_df)
    merged_df = merged_df.drop_duplicates(subset="nome_hotel",keep= False)
    righe_droppate -= len(merged_df)
    merged_df["differenza_prezzo"] = (merged_df["prezzo_x"]-merged_df["prezzo_y"]).abs()
    if merged_df['prezzo_x'].sum() < merged_df['prezzo_y'].sum():
        prezzi_inferiori = "Dataframe 1"
    else: prezzi_inferiori = "Dataframe 2"
    dict_df = {}
    dict_df ['coppia_ricerche'] = orario1+" *** "+orario2
    dict_df ['num_differenze_prezzo'] = len(merged_df)
    dict_df ['media'] = merged_df['differenza_prezzo'].mean()
    dict_df ['mediana'] = merged_df['differenza_prezzo'].median()
    dict_df['%_prezzi_diversi'] = len(merged_df)/initial_len
    dict_df['tipo_ricerche'] = df1['user_agent_type'].unique()[0] + "/"+ df2['user_agent_type'].unique()[0]
    dict_df['prezzi_inferiori'] = prezzi_inferiori
    dict_df['righe_duplicate'] = righe_droppate
    list_dict_df.append(dict_df)

df_differenze = pd.DataFrame(list_dict_df)   
df_differenze.sort_values(by=['num_differenze_prezzo','media'],ascending=False)


,coppia_ricerche,num_differenze_prezzo,media,mediana,%_prezzi_diversi,tipo_ricerche,prezzi_inferiori,righe_duplicate
9,03-02-2024---20:31 *** 03-02-2024---20:42,712,8.571629,7.0,0.740125,desktop/desktop,Dataframe 2,0
20,03-02-2024---20:42 *** 03-02-2024---22:10,703,8.150782,7.0,0.736897,desktop/desktop,Dataframe 1,0
21,03-02-2024---20:42 *** 03-02-2024---22:41,697,8.711621,7.0,0.784910,desktop/mobile,Dataframe 1,0
19,03-02-2024---20:42 *** 03-02-2024---21:42,691,8.798842,7.0,0.786121,desktop/mobile,Dataframe 1,0
17,03-02-2024---20:42 *** 03-02-2024---20:53,689,9.001451,7.0,0.775901,desktop/mobile,Dataframe 2,0
18,03-02-2024---20:42 *** 03-02-2024---21:30,678,8.467552,7.0,0.739368,desktop/desktop,Dataframe 1,0
22,03-02-2024---20:42 *** 03-03-2024---13:08,593,9.558179,7.0,0.771131,desktop/mobile,Dataframe 2,0
1,03-02-2024---20:14 *** 03-02-2024---20:42,480,7.931250,7.0,0.727273,mobile/desktop,Dataframe 2,0
23,03-02-2024---20:42 *** 03-03-2024---13:23,450,7.717778,6.0,0.710900,desktop/desktop,Dataframe 1,0
26,03-02-2024---20:53 *** 03-02-2024---22:10,356,14.632022,13.0,0.398210,mobile/desktop,Dataframe 1,0


In [4]:
# Elimino una o più ricerche dal dataframe
new_df = df_differenze.drop(df_differenze[df_differenze['coppia_ricerche'].str.contains("15:22")].index) 
new_df.sort_values(by=['num_differenze_prezzo','media'],ascending=False)

,coppia_ricerche,num_differenze_prezzo,media,mediana,%_prezzi_diversi,tipo_ricerche,prezzi_inferiori,righe_duplicate
16,03-03-2024---14:02 *** 03-03-2024---16:44,362,15.560773,13.0,0.430952,mobile/desktop,Dataframe 1,2
44,03-03-2024---16:00 *** 03-03-2024---16:44,354,13.937853,13.0,0.416961,mobile/desktop,Dataframe 1,2
43,03-03-2024---15:37 *** 03-03-2024---16:44,353,13.985836,13.0,0.417258,mobile/desktop,Dataframe 1,2
12,03-03-2024---14:02 *** 03-03-2024---15:12,350,14.965714,13.0,0.429975,mobile/desktop,Dataframe 1,2
36,03-03-2024---15:12 *** 03-03-2024---15:37,345,14.008696,13.0,0.424877,desktop/mobile,Dataframe 2,2
37,03-03-2024---15:12 *** 03-03-2024---16:00,345,13.982609,13.0,0.424354,desktop/mobile,Dataframe 2,2
28,03-03-2024---14:31 *** 03-03-2024---16:00,328,14.097561,13.0,0.439678,desktop/mobile,Dataframe 2,2
22,03-03-2024---14:16 *** 03-03-2024---16:00,328,13.219512,12.0,0.386792,desktop/mobile,Dataframe 2,2
27,03-03-2024---14:31 *** 03-03-2024---15:37,327,14.131498,13.0,0.438338,desktop/mobile,Dataframe 2,2
32,03-03-2024---15:01 *** 03-03-2024---15:37,326,14.027607,13.0,0.421734,desktop/mobile,Dataframe 2,2


In [5]:
#Visualizza dataframe mobile/mobile o desktop/desktop
df_tipo_uguale = df_differenze[(df_differenze['tipo_ricerche']=="mobile/mobile") | (df_differenze['tipo_ricerche']=="desktop/desktop")]
df_tipo_uguale.sort_values(by="num_differenze_prezzo",ascending= False)

,coppia_ricerche,num_differenze_prezzo,media,mediana,%_prezzi_diversi,tipo_ricerche,prezzi_inferiori,righe_duplicate
13,03-03-2024---14:02 *** 03-03-2024---15:22,523,12.405354,10.0,0.669654,mobile/mobile,Dataframe 2,0
39,03-03-2024---15:22 *** 03-03-2024---15:37,508,10.978346,10.0,0.648787,mobile/mobile,Dataframe 1,0
40,03-03-2024---15:22 *** 03-03-2024---16:00,504,10.980159,10.0,0.647815,mobile/mobile,Dataframe 1,0
5,03-03-2024---13:49 *** 03-03-2024---15:22,442,9.061086,8.0,0.627841,mobile/mobile,Dataframe 2,0
23,03-03-2024---14:16 *** 03-03-2024---16:44,145,18.889655,18.0,0.151992,desktop/desktop,Dataframe 2,0
19,03-03-2024---14:16 *** 03-03-2024---15:12,143,18.426573,17.0,0.156455,desktop/desktop,Dataframe 2,0
18,03-03-2024---14:16 *** 03-03-2024---15:01,130,18.938462,18.0,0.148233,desktop/desktop,Dataframe 2,0
17,03-03-2024---14:16 *** 03-03-2024---14:31,123,19.308943,18.0,0.145218,desktop/desktop,Dataframe 2,0
14,03-03-2024---14:02 *** 03-03-2024---15:37,72,17.819444,12.5,0.073469,mobile/mobile,Dataframe 2,0
15,03-03-2024---14:02 *** 03-03-2024---16:00,72,17.555556,12.0,0.073469,mobile/mobile,Dataframe 2,0


In [6]:
# Visualizza il numero di ricerche mobile e desktop
mobile = 0
desktop = 0
for i in df_list:
    df1 = df[df["TIMESTAMP"]==i]
    stringa = df1['user_agent_type'].unique()[0]
    if "mob" in stringa:
        mobile +=1
    else: desktop+=1
    print(df1['user_agent_type'].unique())

print("Mobile:", mobile,"Desktop:",desktop)

['mobile']
['mobile']
['desktop']
['desktop']
['desktop']
['desktop']
['mobile']
['mobile']
['mobile']
['desktop']
Mobile: 5 Desktop: 5
